In [1]:
from IPython.display import Image

## Problème 1

En supposant que le corps est fait d'eau pure et suite à la consultation des données XCOM du NIST, nous avon obtenus les coefficients d'atténuation suivant
\begin{align}
\mu_c = 0,1929\ \text{cm$^2$/g}, \ \ \ \ \mu_{n-c} = 0,1824\ \text{cm$^2$/g}.
\end{align}
Le premier indique l'atténuation de l'eau à 70 keV dans un contexte de diffusion chérente tandis que le deuxième est lors d'une diffusion non cohérente. On metionne dans [1] que la diffusion est cohérente lorsque l'énergi du rayonnement incident sur la cible est sous l'énergie d'ionisation de l'atome. Ainsi, dans le cas de l'eau, ayant énergie d'ionisation atomique de près de 13 eV [2], un rayonnement de 70 keV résulte en une diffusion non cohérente. 

L'atténuation de l'intensinté d'un faisceau est donnée par 
\begin{align}
\frac{I}{I_0} = exp(-\int \mu(x) dx).
\end{align}
Or, dans le cas de l'eau, l'atténuation est constante sur le trajet optique. Ainsi, on peut calculer la portion de rayonnement résultante de l'atténuation comme suit
\begin{align}
\frac{I}{I_0} = e^{-\mu l}
\end{align}

Il est à noter que la masse volumique de l'eau étant de 1 g/cm$^3$, nous avons les même grandeurs d'atténuation que ci-haut pour des unités de cm$^{-1}$. 

Bref, 
\begin{align}
 \frac{I}{I_0}\bigg|_{l=5cm} = 0.402, \ \ \ \ \frac{I}{I_0}\bigg|_{l=20cm} = 0.0260
\end{align}

## Reconstruction par rétroprojection

Nous devons faire la reconstruction d'une tomographie à partir d'un sinogramme, ce dernier comprenant l'atténuation en fonction de la position sur le détecteur en fonction de l'angle de la projection. Pour ce faire, une méthode connue sous le nom de voxel-driven, où l'on passe voxel par voxel calculer les contributions des différentes projections, sera utilisée.

## Problème 2

#### Algorithme de reconstruction
Pour créer le laminogramme, nous utilisons une méthode qui consiste en trois étapes. Premièrement, on calcule la distance entre l'axe principal du détecteur et le voxel à l'aide d'une transformation de rotation.
\begin{align}
d_{axe} = (x_0-x)\cos(\theta) - (y_0-y)\sin(\theta)
\end{align}
où x et y représentent les indices de la matrice de voxels, et $x_0, y_0$ sont simplement le point de rotation du système, qu'on pose comme étant le centre de la matrice de voxels. De façon plus imagée, on calcule $d_{axe}$ comme à la figure suivante
<img style="float: center;" src="schema.png" >
Ensuite, on ajoute un facteur de dilatation sur nos distances, ce qui équivaut à une paramétrisation de nos projections. Cela fait en sorte que la projection de notre matrice de voxels va toujours couvrir tout notre espace peu importe l'angle. Le facteur que nous utilisons est de 
\begin{align}
\frac{\sqrt{2}\cdot len(sinogramme)}{2\cdot len(voxels)}
\end{align}
Qui équivaut à la multiplication du facteur de conversion entre les deux tailles et la facteur $\sqrt{2}$, qui s'assure que notre détecteur couvre aussi la matrice en entier lorsqu'elle est à 45$^{\circ}$

Finalement, on converti les distances calculées en indices en les arrondissant à l'entier le plus proche pour ensuite leur aditionner la moitié de la longueur du sinogramme afin d'obtenir les bonnes valeurs d'atténuation. Cette addition vient du fait que les distances $d_{axe}$ sont intialement positives et négatives. Il est important de noter que toutes ces étapes peuvent se faire sans l'usage de boucles for, et qu'implémenter le slicing ici peut augmenter la vitesse d'exécution d'une facteur 100 sans changer la solution.

Il en résulte un algorithme qui solutionne 720 projections de 336 pixels dans un laminogramme de 96 voxels dans des temps inférieurs à une seconde.



<img style="float: center;" src="phantom.png" >
<img style="float: center;" src="Backproject_non_filtre.png" >

In [ ]:
import numpy as np
import time

import geometry as geo
import util as util
import CTfiltre as CTfilter

def readInput():
    # lire les angles
    [nbprj, angles] = util.readAngles(geo.dataDir+geo.anglesFile)

    print("nbprj:",nbprj)
    print("angles min and max (rad):")
    print("["+str(np.min(angles))+", "+str(np.max(angles))+"]")

    # lire le sinogramme
    [nbprj2, nbpix2, sinogram] = util.readSinogram(geo.dataDir+geo.sinogramFile)

    if nbprj != nbprj2:
        print("angles file and sinogram file conflict, aborting!")
        exit(0)

    if geo.nbpix != nbpix2:
        print("geo description and sinogram file conflict, aborting!")
        exit(0)

    return [nbprj, angles, sinogram]

def distance_point_droite(x, y, x0, y0, theta):
    dx =  (x0-x)*np.cos(theta) - (y0-y)*np.sin(theta)
    return dx

def backproject():
    
    [nbprj, angles, sinogram] = readInput()
    
    # initialiser une image reconstruite
    image = np.zeros((geo.nbvox, geo.nbvox))
    
    ### option filtrer ###
    #CTfilter.filterSinogram(sinogram)
    ######
    
    # "etaler" les projections sur l'image
    # ceci sera fait de façon "voxel-driven"
    # pour chaque voxel, trouver la contribution du signal reçu
    x0 = y0 = geo.nbvox / 2
    half_sino = sinogram.shape[1] / 2
    for j in range(geo.nbvox): # colonnes de l'image
        #print("working on image column: "+str(j+1)+"/"+str(geo.nbvox))
        for i in range(geo.nbvox): # lignes de l'image
            dist = np.array(distance_point_droite(i, j, x0, y0, angles))
            dist = np.sqrt(1.9) * dist * half_sino / (geo.nbvox)
            index_inf = (half_sino + dist).astype(int)
            x = np.arange(0,len(sinogram))
            image[j, i] += np.sum(sinogram[(x,index_inf)])
            
    return image

#im = backproject()
#plt.imshow(im)
#plt.show()

### Comparaison fantôme-reconstruction
Lorsqu'on compare le fantôme à la reconstruction, on remarque instantanément les erreurs approtées par cette dernière. On remarque dès lors que près des zones de fort contraste, il y a présence d'une faible netteté des contours, comme si la zone de haute atténuation débordait de la frontière. Malgré le fait qu'ils soient quelques peu flous, il est tout de même possible de distinguer les contours des grandes structures de l'image, tel que les deux lobes foncés. Finalement, les zones blanches sont visibles mais peu claires, et les petits points dans les lobes sont complètement invisibles. Sommairement, il est très difficile de distinguer les détails de l'image, et elle ne pourrait pas être utilisée pour effectuer un diagnostique médical.

## Rétroprojection filtrée
Comme le laminogramme n'est pas une solution exacte de la reconstruction TDM, il est nécessaire de filtrer les basses fréquences du sinogramme avant de le reconstruire pour obtenir une meilleure reconstruction.

## Problème 3

#### Implémentation
L'implémentation d'un filtre passe haut est très simple lorsqu'on suit la documentation de la librairie Fast Fourier Tansform (fft) de Numpy. Il suffit de faire la fft de chaque ligne, puis d'en trouver les fréquences avec la fonction fftfreq, pour ensuite exécuter un fftshift sur ceux-ci. On peut alors effectuer l'opération valeur absolue, en faisant attention de prenre le conjugué de cette valeur, puisqu'elle inverse la partie imaginaire de notre transformée de Fourier. Finalement, on fait toutes les transformations inverses et on enregistre le sinogramme filtré. On observe aux figures ci-dessous le sinogramme non filtré ainsi que le sinogramme filtré.



<img style="float: center;" src="Non-filtre.png" >
<img style="float: center;" src="filtre.png" >

In [ ]:
#from reconstruction import laminogram
sinogram = np.loadtxt("data/sinogram-patient.txt")
filtered_sinogram = []
for line in sinogram:
    fftline = np.fft.fft(line)
    fftshift = np.fft.fftshift(fftline)
    freq =  np.fft.fftfreq(len(line), 1)
    freqshift = np.fft.fftshift(freq)
    fftline = np.conj(np.abs(freqshift)*fftshift)
    lineshift = np.fft.ifftshift(fftline)
    newline = np.fft.fft(lineshift)

    filtered_sinogram.append(newline.real)
    
#plt.imshow(sinogram)
#plt.show()

#### Comparaison
On voit que la forme générale du sinogramme reste inchangé après la filtration, mais que, comme on s'y attendait, les zones de basses fréquences ont maintenant une amplitude qui tend vers zéro. De plus, les zones de fort contrastes sont simplement devenues des pics d'effet de bords (i.e. fort pic négatif suivi d'une fort pic similaire mais inversé). Il s'agit donc du problème inverse qu'on avait lors de la reconstruction, et on imagine que les deux vont se contrebalancer lors de la reconstruction. Ce qui explique la baisse de la baseline (i.e. background et zones d'amplitude constantes) est le simple fait que les zones de valeurs continues sont représentées par des basses fréquences dans l'espace de Fourier. Comme le filtre que nous avons appliqué atténue les basses fréquences, il est tout à fait logique que les grandes zones uniformes de notre sinogramme aient une amplitude qui tend vers zéro.

## Problème 4

### Reconstruction
On entre simpement le sinograme filtré dans notre algorithme du numéro 2 et on obtient une reconstruction bien plus près du fantôme qu'initialement, comme on peut le voir à la figure ci-dessous.



<img style="float: center;" src="Backproject_fft_filtre.png" >

#### Comparaison
On voit tout de suite l'amélioration introduite par la filtration du sinogramme. En effet, il est maintenant possible d'identifier facilement les os et les parois internes des poumons. On peut même maintenant voir les os plus minces de la colonne vertébrale du patient, une nette amélioration! Par contre, il y a toujours présence de bruit dans les zones de faible constraste. Effectivement, si on regarde simplement les zones noires entourant le patient, on remarque un bruit de hautre fréquence. Ce bruit est présent un peu partout dans l'image, et est plus intense au centre des grandes zones uniformes. Ceci est dérangeant si l'on essaie d'identifier les structures dans les poumons du patient, celles-ci étant visibles dans le fantôme. En effet, dans la reconstruction, il est facile de les méprendre pour du bruit, et certaines ne sont simplement pas visibles.

## Problème 5
On tente d'améliorer davantage en faisant l'interpolation linéaire des des pixels adjacents du détecteur pour obtenir une valeur plus précise d'atténuation. Comment l'image s'est-elle améliorée?

### Implémentation
Comme on ne veut pas nuire au temps d'exécution de notre algorithme, il faut implémenter l'interpolation linéaire tout en faisant du slicing. Pour ce faire, on prend les indices supérieurs (sup) et inférieurs (inf) en remplaçant notre arrondis par un np.ceil() et un np.floor(), qui arrondissent vers le haut et le bas respectivement, peu importe le reste. On prend aussi en note ce reste avec l'opération $dist\% 1$ avec "$\%$" représentant l'opération modulo et $dist$ notre $d_{axe}$. Finalement, on utilise ces résultats pour faire notre interpolation, qui se traduit par la ligne de code:

In [ ]:
image[j,i] += np.sum(reste*(sup-inf) + inf)

où, si on pense à une fonction linéaire $ax + b$, reste resprésente $x$, (sup-inf) représente $a$ et inf représente $b$. Il en résulte alors une interpolation linéaire entre les deux pixels et on observe le résultat à la figure suivante

<img style="float: center;" src="Laminogram.png" >

In [ ]:
def laminogram():
    
    [nbprj, angles, sinogram] = readInput()

    # initialiser une image reconstruite
    image = np.zeros((geo.nbvox, geo.nbvox))

    x0 = y0 = geo.nbvox/2
    half_sino = sinogram.shape[1]/2
    for j in range(geo.nbvox): # colonnes de l'image
        #print("working on image column: "+str(j+1)+"/"+str(geo.nbvox))
        for i in range(geo.nbvox): # lignes de l'image
                #rotation et translation de l'espace
                dist = np.array(distance_point_droite(i, j, x0, y0, angles))
                dist = np.sqrt(1.9) * dist * half_sino / (geo.nbvox)
                index_inf = (np.floor(half_sino + dist)).astype(int)
                index_sup = (np.ceil(half_sino + dist)).astype(int)
                residue = dist%1
                x = np.arange(0, len(sinogram))
                inf = sinogram[(x, index_inf)]
                sup = sinogram[(x, index_sup)]
                image[j,i] += np.sum(residue*(sup-inf) + inf)
                
    return image

#im_ = laminogram()
#plt.imshow(im_)
#plt.show()

#### Comparaison
On voit instantanément une nette amélioration de la qualité de l'image. En effet, les artéfacts de haute fréquences semblent avoir complétement disparus, et laissent place à de faibles erreurs continues suivant les sections de haut contraste. Bien que ces erreurs semblent quasiment négligeables, les petites structures dans les poumons ne sont pas parfaitement représentées si l'on compare la reconstruction au fantôme fourni. Par exemple, il est possible de les détecter et de les positionner, mais on ne peut pas calculer leur volume de façon rigoureuse avec cette méthode.

Aussi, les calculs supplémentaires doublent presque le temps de calcul, puisqu'on rajoute une quelques itération pour chaque voxel. Si on moyenne le temps d'exécution sur 20 exécutions du programme, on obtient un temps de $0.81$ s pour l'exécution de l'algorithme normal, et de $1.39$ s pour l'exécution avec interpolation linéaire. Nous avons donc un grand gain en qualité d'image pour un facteur de décélération de $1.73$.

## Problème 6

### Implémentation

Pour produire un méthode de reconstruction par tranches de Fourier, on commence par calculer la transformée de Fourier unidimensionnelle pour chacune des lignes de notre sinogramme tout en faisant les "fftshifts" appropriés. On passe alors à l'algorithme de reconstruction.

Lors de l'implémentation d'une telle méthode, il est premièrement nécessaire de transformer les coordonnées cartésiennes de nos voxels en coordonnées polaires centrées sur le centre de rotation du détécteur, soit le centre de notre matrice de voxels. Pour ce faire, on procède une simple transformation polaire 
\begin{align}
r = \sqrt{x^2 + y^2}\\
\theta = arctan\left(\frac{y}{x}\right)
 \end{align}
Notons que nous avons utilisé la fonction arctan2 de Numpy qui s'assure un résultat allant de $0$ à $2\pi$. On implémente alors cette transformation dans deux boucles "for" parcourant tous les voxels en séquence.
 
On a ensuite besoin de trouver quelles tranches de sinogramme transformé seront utilisées pour calculer la valeur du voxel. Pour ce faire, on trouve l'angle de la matrice d'angles tel que sa différence absolue avec l'angle du voxel (calculé précédemment) est minimale. Il s'agit donc de son plus proche voisin. On obtient par la suite l'indice de cet angle dans la matrice d'angles, et on sélectionne alors la ligne associé à ce dernier dans le sinogramme transformé.\\
 
On peut alors transformer le rayon obtenu plus haut en indexe sur la ligne, en utilisant une fonction linéaire. En effet, l'indice décroit quand le rayon croît, et il est nécessaire de créer une relation $-a\cdot r + b$ pour calculer nos indices. La pente correspond au facteur de dilatation du laminogramme, et b est simplement la moitié de la longueur de la ligne.  Nous avons alors un indice partiel (pas un entier), dont nous pouvons interpoler linéairement la valeur avec le même segment de code qu'au numéro précédent.\\
 
La matrice résultant de notre processus correspond alors à la transformée de Fourier 2D de la reconstruction que nous recherchons. On en fait donc la transformée de Fourier inverse 2D avec les "shifts" appropriés pour obtenir l'image voulue.

In [ ]:
def reconFourierSlice():

    [nbprj, angles, sinogram] = readInput()

    image = np.zeros((geo.nbvox, geo.nbvox), 'complex')
    fft_sino = []
    for line in sinogram:
        fft_sino.append(np.fft.fftshift(np.fft.fft(np.fft.ifftshift(line))))

    sinogram = np.array(fft_sino)

    linefunc = lambda x, a, b: a*x+b
    testat = []
    for j in range(geo.nbvox):  # colonnes de l'image
        print("working on image column: " + str(j + 1) + "/" + str(geo.nbvox))
        for i in range(geo.nbvox):

            x, y  =  -(j - geo.nbvox/2),(i - geo.nbvox/2)
            r = np.linalg.norm([x,y])#/factor
            if r == 0:
                #continue
                r = 1e-20
            theta = np.arctan2(x,y)%(2*np.pi)
            index = np.argmin(np.abs(theta- angles))
            line = sinogram[index]
            print(len(line), geo.nbvox)
            rindex = linefunc(r, -1.25, 168)
            if rindex < 0:
                continue

            rinf = (np.floor(rindex)).astype(int)
            rsup = (np.ceil(rindex)).astype(int)
            reste = rindex%1
            inf = line[rinf]
            sup = line[rsup]
            image[j, i] += (reste * (sup - inf) + inf)
            #image[j, i] += (sup+ inf)/2
            testat.append(rindex)

    image_ifft = np.fft.fftshift(np.fft.ifft2((np.fft.ifftshift(((image))))))
    return image_ifft

## Problème 7

Suite à l'implémentation de la fonction de reconstruction par tranches de Fourier présentée au numéro 6, nous obtenons le résultat suivant pour une taille de voxel de 0.4 et un nombre de 96.

<img style="float: center;" src="Low-Res.png" >

On distingue donc relativement bien les différentes parties du torse. Par contre, on remarque la présence d'artéfacts de reconstruction qui proviennent de l'échantillonnage. Ceux-ci seront discuté au numéro 8. Bref, nous en convenons que l'image obtenue par la méthode présentée au numéro 6 donne un résultat convaincant, i.e. on a bel et bien un torse, mais celle-ci ne pourrait pas être utilisée pour faire un diagnostic adéquat étant donné la présence des erreurs de reconstruction du au faible échantillonage dans l'espace de Fourier. On obtient le résulta à de la figure suivante dans un temps de calcul de 0.92±0.02 secondes.

## Problème 8

En utilisant la méthode présentée au numéro 6, mais cette fois avec une dimension des voxels de 0.2 et un nombre de 192, on obtient l'image suivante

<img style="float: center;" src="Hi-Res.png" >

Tout d'abord, on peut remarquer qu'une augmentation du nombre de voxel mène à un meilleur contraste pour les petites structures, mais que des artéfacts de reconstruction sont toujours présents. On classifie d'atréfacts les grandes lignes noires semi-circulaires apparaissant dans notre image. Ces dernières proviennent d'effets de bords dans la transformée de Fourier, qui sont supposés être à l'extérieur de l'image mais viennent qu'à boucler dû à la nature périodique de l'espace de Fourier.

On note une nette amélioration de la qualité de l'image, et la disparition des bruit de haute fréquence (de l'ordre de deux pixels). Cette amélioration provient du fait que plus de pixels dans l'espace de Fourier implique une plus grande fréquence d'échantillonage possible lors de la reconstruction. En effet, si la résolution d'un système augmente avant qu'on en fasse la transformée de fourier (ou la transformation inverse), non seulement la résolution après transformation va augmenter, mais une plus grande gamme de fréquences peuvent être prises en compte dû à l'augmentation de la fréquence d'échantillonage. La présence des abhérations préiodiques des périodes de deux pixel (fréquence de Nyquist) dans la figure à faible résolution ne fait que nous indiquer qu'on ne couvre pas un assez grande plage de fréquences pour représenter les contrastes (fonction porte) présents dans l'image. Ceci est confirmé par leur disparition dans l'image à plus haute résolution.

Étant donné la meilleure résolution, un diagnostic effectué à partir de cette image serait capable de cerner la position des objets ou des tumeurs, le cas échéant. Il semblerait donc légitime de dire que cette image pourrait être utilisée lors d'un diagnostic, en sachant toutefois que les traces semi-circulaires sont des artéfacts.

## Problème 9

Suite à l'implémentation de différentes méthodes de reconstruction tomographiques, il semble naturel de vouloir comparer leur temps d'exécution. Pour ce faire, nous avons exécuté 20 fois chacune ces méthodes, en avons pris la moyenne et avons associé une incertitude correspondant à 2 écarts types de la distribution de temps d'exécution obtenus. Nous avons donc que la méthode la plus lente est la méthode des tranches de Fourier à haute résolution (i.e. 192 voxels ayant un taille de 0.2) avec un temps moyen de calcul de 2.47±0.01 secondes, s'en suit la méthode de rétroprojection avec un temps de calcul moyen de 1.18±0.01 secondes. Bref, la méthode la plus efficace s'avère être celle des tranches de Fourier à basse résolution (i.e. 96 voxels ayant un taille de 0.4) avec un temps d'exécution de 0.92±0.02 secondes. En ce qui concerne les boucles for imbriquées lors de ces méthode, chacune d'entre elle en possède 2, soit une pour les lignes et l'autre pour les colonnes de la matrice de voxels. Un gestion adéquate des angles par slicing est utilisée pour la méthode de rétroprojection tandis qu'un seul angle est considéré pour chaque point de la reconstruction pour la reconstruction par tranches de Fourier. De plus, la méthode de correspondance sinogram-reconstruction n'est pas la même pour la rétroprojection et pour les tranches de Fourier (elle contient deux fft), d'où l'augmentation du temps de calcul.

## Problème 10

Il n'est pas très difficile d'implémenter l'interpolation bilinéaire dans notre algorithme. Cette méthode consiste en fait à multiplier chacun des quatres points voisins à leur portion d'aire alterne-interne associée, comme on peut le voir dans l'image suivante.

<img style="float: center;" src="bili.png" >

En effet, comme nous faisons déja une interpolation linéaire sur les rayons, il ne faut qu'implémenter la même chose pour les angles et faire les multiplications appropriées.\\

Par contre, comme nous sommes en coordonnées polaires, il faut utiliser les formules de calcul d'aires de section de cercle, soit
\begin{align}
A = (r_f^2 - r_i^2)\cdot \Delta\theta/2
\end{align}

Il en résulte un temps de calcul légèrement plus long pour obtenir un patron de Fourier plus précis, dû à l'interpolation en $\theta$.


## Références
[1] Radiopedia, S. Price et al. https://radiopaedia.org/articles/coherent-scattering 

[2] WebBook de Chimie du NIST https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Mask=20